In [1]:
#import pandas as pd
import json
import numpy as np
from tqdm import tqdm_notebook as tqdm
import uuid
import weaviate 
from weaviate.batch import Batch 
import helper
import weaviate



In [2]:
from weaviate.wcs import WCS
my_credentials = weaviate.auth.AuthClientPassword(username='yilin2718@gmail.com', password='KC3XUckyWW2TiBW')
my_wcs = WCS(my_credentials)

In [3]:
cluster_name = 'product-round2'
weaviate_url = my_wcs.create(cluster_name=cluster_name)


In [4]:
client = weaviate.Client(weaviate_url)
weaviate_url



'https://product-round2.semi.network'

In [6]:
# helper function
def prettify(json_dict): 
    print(json.dumps(json_dict, indent=2))

def generate_uuid(class_name: str, identifier: str,
                  test: str = 'teststrong') -> str:
    """ Generate a uuid based on an identifier
    :param identifier: characters used to generate the uuid
    :type identifier: str, required
    :param class_name: classname of the object to create a uuid for
    :type class_name: str, required
    """
    test = 'overwritten'
    return str(uuid.uuid5(uuid.NAMESPACE_DNS, class_name + identifier))


In [7]:
client.schema.delete_all() # delete all classes

In [8]:
import weaviate

# skipped product "brand", it can be it's own class, for simplicity, skip it for now 
# skipped product "category" list of strings (text?) not sure how to type it yet, or if category should be it's own class?
# skipped "similar_item"

schema = {
    "classes": [
     {
            # name of the class
            "class": "Product",
            # class properties
            "properties": [
                {
                    "name": "asin",
                    "dataType": ["string"]
                },
                {
                    "name": "title",
                    "dataType": ["text"]
                },
                {
                    "dataType": ["number"],
                    "description": "The price product in dollars",
                    "name": "price"
                },
                {
                    "dataType": ["text"],
                    "name": "productDescription",
                    "description": "description of product"

                },
                {
                    "dataType": ["string"],
                    "name": "mainCat",
                    "description": "main category of the product in amazon "
                },
                {
                    "dataType": [
                        "Review"
                    ],
                    "description": "review of this product",
                    "name": "ofReview"
                }

            ]
        },{
            "class": "Review",
            "description": "A Review class to store the review information",
            "properties": [
                {
                    "dataType": [
                        "text"
                    ],
                    "description": "review text",
                    "name": "reviewText",
                },
                {
                    "dataType": [
                        "string"
                    ],
                    "description": "product id from amazon",
                    "name": "asin",
                },
                {
                    "dataType": [
                        "text"
                    ],
                    "description": "reviewer provided summary of review",
                    "name": "summary",
                },
                {
                    "dataType": [
                        "Product"
                    ],
                    "description": "The product of the review",
                    "name": "ofProduct"
                }
            ]
        }
    ]
}
client.schema.create(schema)

prettify(client.schema.get())

In [9]:

# load all metadata
count = 0
data = []
with open('data/sample_meta_Home_and_Kitchen.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))
        count = count + 1
        if count >= 100:
          break

/var/folders/4c/x3415rcx4pb9drxwpfyzk32h0000gn/T/ipykernel_37446/3184668384.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(f):


0it [00:00, ?it/s]

In [10]:

productData = []

for i in range(0, 100):
    hasprice="price" in data[i]
    hastitle="title" in data[i]
    hasdescription="description" in data[i]
    hasmaincat='main_cat' in data[i]
    #print(i,  (hasprice and hastitle and hasdescription))
    if (hasprice and hastitle and hasdescription and hasmaincat)==True:
        productData.append(data[i])
    else:
        continue


for i in range(0, len(productData)): 
   productData[i]['description']=" ".join(productData[i]['description'])
   cleaned_text = productData[i]['description'].replace("\\", "")
   productData[i]['description']=cleaned_text
   productData[i]['price']=productData[i]['price'].replace("$","").replace(",","")
   productData[i]['price']=float(productData[i]['price'])




In [41]:
len(productData)

17

In [42]:
productData[10]

{'category': ['Home & Kitchen', 'Vacuums & Floor Care'],
 'description': 'Hoses- Electric SES118 Miele S500-600 direct connect. Electric Hose- A required component for the use of all electric powerbrushes.',
 'title': 'Miele SES118 Vacuum Cleaner Direct-Connect Electric Hose',
 'brand': 'Miele',
 'rank': '>#720,580 in Home & Kitchen (See Top 100 in Home & Kitchen)>#13,142 in Home & Kitchen > Vacuums & Floor Care',
 'main_cat': 'Amazon Home',
 'price': 135.0,
 'asin': 'B0002U37E4'}

In [11]:
cand_asin = set([d['asin'] for d in productData])

print(len(cand_asin))

12


for i in range(0, len(productData)): 
    print(productData[i]['asin'])

In [43]:
productData[2]['price']

5.1

In [44]:
productData[14]['price']

5.1

In [47]:
reviews = []


In [12]:
# align products with reviews
from collections import defaultdict
reviews = defaultdict(list)
with open('data/sample_Home_and_Kitchen_5.json', 'r') as f:
    for l in f:
        r = json.loads(l)
        a = r['asin']
        if a in cand_asin:
            reviews[a].append(r)

In [13]:
i=12
productid=productData[i]['asin']
print(productid)
reviews[productid][1]['summary']

B00002N62Y


'Great price - OEM belts!'

In [18]:
len(productData)

17

In [14]:
from weaviate.batch import Batch # for the typing purposes


def add_product(batch: Batch, product_data: dict) -> str:
    product_object = {
        'asin': product_data['asin'],
        'title': product_data['title'],
        'main_cat': product_data['main_cat'],
        'description': product_data['description'],             
        #'feature': product_data['feature'] if 'feature' in product_data else '',
        'price': product_data['price']
    }
    # generate an UUID for the Author
    product_id = generate_uuid('product', product_data['title']+product_data['asin'])
   
    # add article to the batch
    batch.add_data_object(  
        data_object=product_object,
        class_name='Product',
        uuid=product_id
    ) 
    return product_id


def add_review(batch: Batch, review_data: dict) -> str:
    review_object = {
        'asin': review_data['asin'] ,
        'reviewText':review_data['reviewText'],               
        'summary': review_data['summary']
    }
    # generate an UUID for the review
    review_id = generate_uuid('review', review_data['reviewText'])

    batch.add_data_object(  
        data_object=review_object,
        class_name='Review',
        uuid=review_id
    )
    return review_id


def add_references(batch: Batch, product_id: str, review_id: str)-> None:
    # add references to the batch
    ## Review -> Product
    batch.add_reference(  
        from_object_uuid=review_id,
        from_object_class_name='Review',
        from_property_name='ofProduct',
        to_object_uuid=product_id
    )
    
    ## Product -> Review 
    batch.add_reference(  
        from_object_uuid=product_id,
        from_object_class_name='Product',
        from_property_name='ofReview',
        to_object_uuid=review_id
    )


from tqdm import trange

for i in trange(0, 17):
    product_id = add_product(client.batch, productData[i])
    if i % 20 == 0:
        # submit the objects from the batch to weaviate
        client.batch.create_objects()
         
status_objects = client.batch.create_objects()
client.batch.flush()

In [15]:
list = [ 5, 7]
   
# Using for loop
for i in [5, 7]: #trange(0,6):
    p_asid=productData[i]['asin']
    print(p_asid)
    print(reviews[p_asid][0])

B0000D83BR
{'overall': 1.0, 'verified': True, 'reviewTime': '11 6, 2017', 'reviewerID': 'A2JEXZB73CRPJS', 'asin': 'B0000D83BR', 'reviewerName': 'Norman P. Rounds', 'reviewText': 'Very cheaply made. It hardly picks up anything. A piece of crap!', 'summary': 'A piece of crap!', 'unixReviewTime': 1509926400}
B0002EB670
{'overall': 5.0, 'verified': True, 'reviewTime': '03 29, 2010', 'reviewerID': 'A2WZMVUM0DM520', 'asin': 'B0002EB670', 'reviewerName': 'Janice Jennings', 'reviewText': 'For some reason, the belts do not last very long, so keep extras on hand!', 'summary': 'Keep extras', 'unixReviewTime': 1269820800}


In [21]:
i=5
p_asid=productData[i]['asin']
print(p_asid)
reviews[p_asid][0]
#print(reviews[p_asid])

B0000D83BR


{'overall': 1.0,
 'verified': True,
 'reviewTime': '11 6, 2017',
 'reviewerID': 'A2JEXZB73CRPJS',
 'asin': 'B0000D83BR',
 'reviewerName': 'Norman P. Rounds',
 'reviewText': 'Very cheaply made. It hardly picks up anything. A piece of crap!',
 'summary': 'A piece of crap!',
 'unixReviewTime': 1509926400}

productData[5]

In [27]:
len(productData)

151

In [16]:
from tqdm import trange

for i in trange(1, 10): 
    product_id = add_product(client.batch, productData[i])

    p_asid=productData[i]['asin']
    print(p_asid)


    for review in reviews[p_asid]:
        # add review to the batch
        review_id = add_review(client.batch, review)

        # add cross references to the batch
        add_references(client.batch, product_id=product_id, review_id=review_id)


    if i % 20 == 0:
        # submit the objects from the batch to weaviate
        client.batch.create_objects()
        # submit the references from the batch to weaviate
        client.batch.create_references()

         
status_objects = client.batch.create_objects()
status_references = client.batch.create_references()
client.batch.flush()

100%|██████████| 9/9 [00:00<00:00, 175.77it/s]


B00002N8CX
B000050686
B000050HCV
B00005V9E3
B0000D83BR
B0001AOE5Q
B0002EB670
B0002U352S
B0002U373A


In [17]:
result = client.query.get(class_name='Product', properties="title")\
    .do()
print(f"Number of reviews returned: {len(result['data']['Get']['Product'])}")
#result

Number of reviews returned: 9


In [ ]:
Below are old ones 

from weaviate.batch import Batch # for the typing purposes
from weaviate.util import generate_uuid5 # old way was from weaviate.tools import generate_uuid
import helper
from weaviate.util import generate_uuid5 # old way was from weaviate.tools import generate_uuid


def add_product(batch: Batch, product_data: dict) -> str:
    product_object = {
        'asin': product_data['asin'] if 'asin' in product_data else '',
        'title': product_data['title'] if 'title' in product_data else '',
        'main_cat': product_data['main_cat'] if 'main_cat' in product_data else '',
        'description':product_data['description'] if 'description' in product_data else '',               
        'feature': product_data['feature'] if 'feature' in product_data else '',
        'price': float(product_data['price'].replace("$","").replace(",","")) if 'price' in product_data else float("NaN")
    }
    # generate an UUID for the Author
    product_id = generate_uuid('product', product_data['title']+product_data['asin'])
   
    # add article to the batch
    batch.add_data_object(  # old way was batch.add(...)
        data_object=product_object,
        class_name='Product',
        uuid=product_id
    )
    
    return product_id

def add_review(batch: Batch, review_data: dict) -> str:
    review_object = {
        'asin': review_data['asin'] if 'asin' in review_data else '',
        'reviewText':review_data['reviewText'] if 'reviewText' in review_data else '',               
        'summary': review_data['summary'] if 'summary' in review_data else ''
    }
    # generate an UUID for the review
    review_id = generate_uuid('review', review_data['reviewText'])
    #review_id = generate_uuid5(review)
    # add article to the batch
    batch.add_data_object(  # old way was batch.add(...)
        data_object=review_object,
        class_name='Review',
        uuid=review_id
    )
    
    return review_id


def add_references(batch: Batch, product_id: str, review_id: str)-> None:
    # add references to the batch
    ## Review -> Product
    batch.add_reference(  
        from_object_uuid=review_id,
        from_object_class_name='Review',
        from_property_name='ofProduct',
        to_object_uuid=product_id
    )
    
    ## Product -> Review 
    batch.add_reference(  
        from_object_uuid=product_id,
        from_object_class_name='Product',
        from_property_name='hasReviews',
        to_object_uuid=review_id
    )

from tqdm import trange

for i in trange(1, 1000):
    review_id = add_review(client.batch, reviewData[i])         
      
status_objects = client.batch.create_objects()
client.batch.flush()

In [ ]:
result = client.query.get(class_name='Review', properties="reviewText")\
    .do()
print(f"Number of reviews returned: {len(result['data']['Get']['Review'])}")
result

Number of reviews returned: 100


{'data': {'Get': {'Review': [{'reviewText': "I am very pleased with my new Dyson. The roller ball system really does make it easier to navigate the unit around the house. It is a tad smaller than I'm used to but that smaller=lighter and I'm getting used to it. I had literally vacuumed the day before with my old Hoover and then vacuumed again with the new dyson the day it arrived I had to empty the contained 4 times! The carpet looked amazing. This unit is a little expensive but well worth the money, no more bargain vacuum cleaners for me!"},
    {'reviewText': "I have an old electrolux and was afraid I wouldn't be able to find the bags, but I was wrong.  I ordered these and they are exactly right.  Thank you"},
    {'reviewText': 'So I guess I would give this 4.5 stars.  It isn\'t perfect, but I actually like it more than any vacuum I have ever owned.  I bought it in 2010 when my German shepherd was a puppy.  We already had a golden retriever and the two together were making more fur t

In [458]:
result = client.query.get(class_name='Product', properties="price")\
    .do()
print(f"Number of reviews returned: {len(result['data']['Get']['Product'])}")
result

Number of reviews returned: 2


{'data': {'Get': {'Product': [{'price': 3.76}, {'price': 30.44}]}}}

from tqdm import trange

for i in trange(1, 3025):
    product_id = add_product(client.batch, productData[i])         
    client.batch.create_objects()
status_objects = client.batch.create_objects()
client.batch.flush()